In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(2012)

# Configure visual settings:
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10.0, 8.0) 
plt.style.use(['bmh'])

# Load the dataframe
data = pd.read_pickle('assets/engineered_data1.p')

In [2]:
data.iloc[0,:]

help_packing                                                            Yes
boxes                                                                    45
furniture                 Sectional Sofa (11'6"x7'10"x2'9"), Desk (5'x2'...
reference                       Good reviews online. Seen truck everywhere.
mention                   Would like to receive two estimates: one for p...
est_hours                                                              3.25
truck                                                              Behemoth
num_movers                                                                3
rate                                                                    200
travel_fee                                                                1
min_hours                                                                 3
loc1.lengthOfWalkOptID                                                    6
loc1.sqFt                                                               800
loc1.elevato

In [3]:
data[['loc1.fullAddress','loc1.lat','loc1.lng','loc1.geocodedAddress',
  'loc2.fullAddress','loc2.lat','loc2.lng','loc2.geocodedAddress']].head()


,loc1.fullAddress,loc1.lat,loc1.lng,loc1.geocodedAddress,loc2.fullAddress,loc2.lat,loc2.lng,loc2.geocodedAddress
0,"3201 Landover Street #1723, Alexandria, VA, 22202",38.833873,-77.062375,"3201 Landover St #1723, Alexandria, VA 22305, USA","815 18th Street S #205, Arlington, VA, 22202",38.857344,-77.060368,"815 18th St S #205, Arlington, VA 22202, USA"
1,"1437 Spring Road NW, Washington, DC, 20009",38.937674,-77.034692,"1437 Spring Rd NW, Washington, DC 20010, USA","1688 Euclid Street NW, Washington, DC, 20009",38.923162,-77.038777,"1688 Euclid St NW, Washington, DC 20009, USA"
3,"Metropolitan, 7620 Old Georgetown Road Apt 812...",38.985762,-77.096627,,"Courts of Devon, 505 Main Street, Apt 232, Gai...",39.120716,-77.235182,
4,"505 E Braddock Rd. #107, Alexandria, VA, 20165",38.812218,-77.054390,"505 E Braddock Rd #107, Alexandria, VA 22314, USA","47587 Reer Terrace, Sterling, VA, 20165",39.039813,-77.352041,"47587 Reef Terrace, Sterling, VA 20165, USA"
7,"3220 Connecticut Ave NW, Washington, DC, 20002",38.933215,-77.057620,"3220 Connecticut Ave NW, Washington, DC 20008,...","1656 Rosedale St NE, Washington, DC, 20002",38.897448,-76.980548,"1656 Rosedale St NE, Washington, DC 20002, USA"


In [4]:
import requests

url = 'https://www.python.org/'
r = requests.get(url)

assert type(r.text) == str

In [5]:
google_api_key = open('google_api_key.txt', mode='r').read()


import googlemaps
from datetime import datetime

In [6]:
gmaps = googlemaps.Client(key=google_api_key)

#%%
origins = ['19 Rock Creek Church Rd NW, Washington, DC 20011']
destinations = ['1133 15th St NW, Washington, DC']


# distance_results = gmaps.distance_matrix(origins, destinations, mode="driving")

import time

In [7]:
def get_driving_time(row, sleep=True):
    origin = row['loc1.geocodedAddress']
    destination = row['loc2.geocodedAddress']
    if origin == '':
        if row.isnull()['loc1.lat'] or row.isnull()['loc1.lng']:
            origin = row['loc1.fullAddress']
        else:
            origin = row['loc1.lat'],row['loc1.lng']
    if destination == '':
        if row.isnull()['loc2.lat'] or row.isnull()['loc2.lat']:
            destination = row['loc2.fullAddress']
        else:
            destination = row['loc2.lat'],row['loc2.lng']
    
    if sleep:
        time.sleep(np.random.randint(1,2))
    try:
        distance_results = gmaps.distance_matrix(origin, destination, mode="driving")
        seconds = distance_results['rows'][0]['elements'][0]['duration']['value']
    except:
        return(np.nan)
    return(seconds)

In [8]:
data = data.drop_duplicates()

In [9]:
get_driving_time(data.iloc[0,:], sleep = False)

587

In [10]:
test = [1,2,3]
import pickle
with open('test.p', 'wb') as f:
    pickle.dump(test, f)

In [11]:
with open('test.p', 'rb') as f:
    test2 = pickle.load(f)
test2

[1, 2, 3]

In [12]:
# distance_list = []
# for i in range(0,data.shape[0],100):
#     start = i
#     batch_list = [get_driving_time(data.iloc[x,:], sleep = False) for x in range(start, min(data.shape[0], i+100))]
#     distance_list += batch_list
#     print(i)
#     if i % 1000 == 0:
#         print('Pickling list')
#         with open('distance_list.p', 'wb') as f:
#             pickle.dump(distance_list, f) 
#     time.sleep(3)

# with open('distance_list.p', 'wb') as f:
#     pickle.dump(distance_list, f) 

In [13]:
with open('distance_list.p', 'rb') as f:
    distance_list = pickle.load(f)
assert len(distance_list) == data.shape[0]

In [14]:
data['driving_distance'] = distance_list

In [15]:
# If driving distance is greater than 3 hours, I must have done something wrong:
data.loc[data['driving_distance'] > 3 * 60 * 60, 'driving_distance'] = data['driving_distance'].median()

data['driving_distance'].fillna(data['driving_distance'].median(), inplace=True)

# I should revisit this. IE, if both addresses are in DC then set driving distance equal to half an hour

In [17]:
data.to_pickle('assets/engineered_data2.p')